# Import Package

In [ ]:
!git clone --recurse-submodules https://github.com/dmlc/dgl.git

Cloning into 'dgl'...
remote: Enumerating objects: 25089, done.
remote: Counting objects: 100% (639/639), done.
remote: Compressing objects: 100% (445/445), done.
remote: Total 25089 (delta 268), reused 458 (delta 179), pack-reused 24450
Receiving objects: 100% (25089/25089), 15.27 MiB | 20.79 MiB/s, done.
Resolving deltas: 100% (15791/15791), done.
Submodule 'third_party/METIS' (https://github.com/KarypisLab/METIS.git) registered for path 'third_party/METIS'
Submodule 'third_party/cub' (https://github.com/NVlabs/cub.git) registered for path 'third_party/cub'
Submodule 'third_party/dlpack' (https://github.com/dmlc/dlpack.git) registered for path 'third_party/dlpack'
Submodule 'third_party/dmlc-core' (https://github.com/dmlc/dmlc-core.git) registered for path 'third_party/dmlc-core'
Submodule 'third_party/googletest' (https://github.com/google/googletest.git) registered for path 'third_party/googletest'
Submodule 'third_party/libxsmm' (https://github.com/hfp/libxsmm.git) registered for 

In [ ]:
%cd dgl

/content/dgl


In [ ]:
!pip install dgl

     |████████████████████████████████| 4.4 MB 5.1 MB/s 


In [ ]:
%cd /content

/content


In [ ]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


DGL backend not selected or invalid.  Assuming PyTorch for now.
Using backend: pytorch


# Loading Data

In [ ]:
import dgl.data

dataset = dgl.data.CoraGraphDataset()
print('Number of categories:', dataset.num_classes)

Extracting file to /root/.dgl/cora_v2
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
Number of categories: 7


In [ ]:
g = dataset[0] # CoraGraphDataset只有一張圖，所以只讀的到第0筆資料

In [ ]:
# 透過graph._data可取得特徵資料, _可填入：
# e:表示邊的資料
# n:表示點的資料
print('Node features:') 
print(g.ndata)  
print('Edge features:')
print(g.edata)

Node features:
{'train_mask': tensor([ True,  True,  True,  ..., False, False, False]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])}
Edge features:
{}


In [ ]:
g.ndata['feat'].shape

torch.Size([2708, 1433])

# Define a GCN

In [ ]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)  # Forward時, 傳入圖 & 資料的維度(有多少筆特徵)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

# Create the model with given dimensions
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)

# Training

In [ ]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(100):
        # Forward
        logits = model(g, features) # 會自動呼叫model.forward, 其中g作為圖傳入, features則做為資料維度傳入

        # Compute prediction
        pred = logits.argmax(1) # 透過argmax分類

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))

model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)
train(g, model)

In epoch 0, loss: 1.946, val acc: 0.102 (best 0.102), test acc: 0.115 (best 0.115)
In epoch 5, loss: 1.908, val acc: 0.572 (best 0.572), test acc: 0.594 (best 0.594)
In epoch 10, loss: 1.842, val acc: 0.600 (best 0.600), test acc: 0.616 (best 0.607)
In epoch 15, loss: 1.753, val acc: 0.622 (best 0.622), test acc: 0.676 (best 0.676)
In epoch 20, loss: 1.638, val acc: 0.678 (best 0.678), test acc: 0.699 (best 0.699)
In epoch 25, loss: 1.497, val acc: 0.692 (best 0.692), test acc: 0.712 (best 0.712)
In epoch 30, loss: 1.337, val acc: 0.720 (best 0.720), test acc: 0.721 (best 0.721)
In epoch 35, loss: 1.163, val acc: 0.726 (best 0.728), test acc: 0.737 (best 0.735)
In epoch 40, loss: 0.987, val acc: 0.724 (best 0.728), test acc: 0.735 (best 0.735)
In epoch 45, loss: 0.818, val acc: 0.726 (best 0.728), test acc: 0.748 (best 0.735)
In epoch 50, loss: 0.666, val acc: 0.734 (best 0.734), test acc: 0.755 (best 0.755)
In epoch 55, loss: 0.535, val acc: 0.744 (best 0.744), test acc: 0.757 (best 0